In [ ]:
# Food Delivery Data Analysis
# This notebook loads, cleans, and analyzes the food delivery dataset.

import pandas as pd
import json
import re
import numpy as np


## 1. LOAD THE DATA


# Load Orders
orders_df = pd.read_csv('orders.csv')
print(f"Data Loaded: Orders {orders_df.shape}")

# Load Users
with open('users.json', 'r') as f:
    users_df = pd.DataFrame(json.load(f))
print(f"Data Loaded: Users {users_df.shape}")

# Load Restaurants (from SQL file)
restaurants_data = []
with open('restaurants.sql', 'r') as f:
    for line in f:
        if line.startswith('INSERT INTO restaurants VALUES'):
            match = re.search(r'\((.*?)\);', line)
            if match:
                parts = [p.strip().strip("'") for p in match.group(1).split(',')]
                if len(parts) >= 4:
                    restaurants_data.append({
                        'restaurant_id': int(parts[0]),
                        'restaurant_name_master': parts[1],
                        'cuisine': parts[2],
                        'rating': float(parts[3])
                    })
restaurants_df = pd.DataFrame(restaurants_data)
print(f"Data Loaded: Restaurants {restaurants_df.shape}")


## 2. MERGE THE DATASETS


# Merge Orders -> Users
# Logic: We use 'left' join to keep all orders
merged_df = pd.merge(orders_df, users_df, on='user_id', how='left')

# Merge -> Restaurants
merged_df = pd.merge(merged_df, restaurants_df, on='restaurant_id', how='left')

# Convert Date
merged_df['order_date'] = pd.to_datetime(merged_df['order_date'], dayfirst=True)

# Save the final file
merged_df.to_csv('final_food_delivery_dataset.csv', index=False)
print("Success: 'final_food_delivery_dataset.csv' created.")


## 3. PART 1: MULTIPLE CHOICE ANALYSIS


print("\n" + "="*40)
print(" PART 1: ANALYTICAL SOLUTIONS")
print("="*40)

# Q1: Highest Revenue City (Gold)
q1 = merged_df[merged_df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().idxmax()
print(f"1. Highest Gold Revenue City: {q1}")

# Q2: Highest AOV Cuisine
q2 = merged_df.groupby('cuisine')['total_amount'].mean().idxmax()
print(f"2. Highest AOV Cuisine: {q2}")

# Q3: Users with > 1000 Value
user_sums = merged_df.groupby('user_id')['total_amount'].sum()
q3 = (user_sums > 1000).sum()
# Logic: The question asks to select range. 2544 is > 2000.
print(f"3. Count of users > 1000 value: {q3} (Corresponds to option '> 2000')")

# Q4: Best Rating Range
def get_range(r):
    if 3.0 <= r <= 3.5: return '3.0 – 3.5'
    elif 3.6 <= r <= 4.0: return '3.6 – 4.0'
    elif 4.1 <= r <= 4.5: return '4.1 – 4.5'
    elif 4.6 <= r <= 5.0: return '4.6 – 5.0'
merged_df['rating_range'] = merged_df['rating'].apply(get_range)
q4 = merged_df.groupby('rating_range')['total_amount'].sum().idxmax()
print(f"4. Highest Revenue Rating Range: {q4}")

# Q5: Highest AOV City (Gold)
q5 = merged_df[merged_df['membership'] == 'Gold'].groupby('city')['total_amount'].mean().idxmax()
print(f"5. Highest AOV City (Gold): {q5}")

# Q6: Lowest Distinct Restaurants
q6 = merged_df.groupby('cuisine')['restaurant_id'].nunique().idxmin()
print(f"6. Cuisine with lowest distinct restaurants: {q6}")

# Q7: Gold Order Percentage
gold_pct = (len(merged_df[merged_df['membership'] == 'Gold']) / len(merged_df)) * 100
print(f"7. Gold Order Percentage: {round(gold_pct)}%")

# Q8: Highest AOV Restaurant (<20 orders)
# Logic: Filter specific options first, then check conditions
q8_options = ["Grand Cafe Punjabi", "Grand Restaurant South Indian", "Ruchi Mess Multicuisine", "Ruchi Foods Chinese"]
stats = merged_df.groupby('restaurant_name').agg(count=('order_id','count'), aov=('total_amount','mean'))
filtered_stats = stats.loc[stats.index.intersection(q8_options)]
valid_q8 = filtered_stats[filtered_stats['count'] < 20]
q8_ans = valid_q8['aov'].idxmax() if not valid_q8.empty else "None"
print(f"8. Best Restaurant (<20 orders) from options: {q8_ans}")

# Q9: Best Revenue Combo
# Logic: Calculate revenue for specific combinations given
q9_options = [('Gold', 'Indian'), ('Gold', 'Italian'), ('Regular', 'Indian'), ('Regular', 'Chinese')]
combo_rev = merged_df.groupby(['membership', 'cuisine'])['total_amount'].sum()
# Find max among the specific list
best_combo = max(q9_options, key=lambda x: combo_rev.get(x, 0))
print(f"9. Best Revenue Combination: {best_combo[0]} + {best_combo[1]}")

# Q10: Best Quarter
merged_df['quarter'] = merged_df['order_date'].dt.quarter.map({1:'Q1', 2:'Q2', 3:'Q3', 4:'Q4'})
q10 = merged_df.groupby('quarter')['total_amount'].sum().idxmax()
print(f"10. Highest Revenue Quarter: {q10}")



## 4. PART 2: NUMERICAL ANALYSIS

print("\n" + "="*40)
print(" PART 2: NUMERICAL CALCULATIONS")
print("="*40)

# 1. Total Gold Orders
num1 = len(merged_df[merged_df['membership'] == 'Gold'])
print(f"1. Total Gold Orders: {num1}")

# 2. Hyderabad Revenue
num2 = round(merged_df[merged_df['city'] == 'Hyderabad']['total_amount'].sum())
print(f"2. Hyderabad Revenue: {num2}")

# 3. Distinct Users
num3 = merged_df['user_id'].nunique()
print(f"3. Distinct Users: {num3}")

# 4. Gold AOV
num4 = round(merged_df[merged_df['membership'] == 'Gold']['total_amount'].mean(), 2)
print(f"4. Gold AOV: {num4}")

# 5. High Rating Orders
num5 = len(merged_df[merged_df['rating'] >= 4.5])
print(f"5. Orders (Rating >= 4.5): {num5}")

# 6. Orders in Top Gold City
top_gold_city = merged_df[merged_df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().idxmax()
num6 = len(merged_df[(merged_df['membership'] == 'Gold') & (merged_df['city'] == top_gold_city)])
print(f"6. Orders in Top Gold City ({top_gold_city}): {num6}")



## 5. PART 3: FILL-IN-THE-BLANKS
print("\n" + "="*40)
print(" PART 3: LOGICAL DERIVATION (FILL IN BLANKS)")
print("="*40)

# 1. Column to join orders and users
# Logic: Find intersection of columns
common_cols = list(set(orders_df.columns) & set(users_df.columns))
print(f"1. Join Column (Orders-Users): {common_cols[0]}")

# 2. Dataset format for restaurants
# Logic: We parsed 'INSERT INTO' statements, which is standard SQL.
print(f"2. Restaurant Data Format: SQL (Deduced from file content parsing)")

# 3. Total rows in final dataset
# Logic: Count rows in merged_df
print(f"3. Total Rows in Merged Dataset: {merged_df.shape[0]}")

# 4. Value for missing user records
# Logic: Check for nulls in a user column (e.g., 'name')
null_check = merged_df['name'].isnull().unique()
print(f"4. Missing Value Representation: {null_check[0]} (NaN)")

# 5. Pandas function used
# Logic: We used the function explicitly in code
print(f"5. Pandas Join Function: merge")

# 6. Membership column origin
# Logic: Check which original dataframe had 'membership'
origin = "users.json" if 'membership' in users_df.columns else "orders.csv"
print(f"6. Membership Column Origin: {origin}")

# 7. Join key for restaurants
# Logic: Find intersection of columns between orders and restaurants
common_rest_cols = list(set(orders_df.columns) & set(restaurants_df.columns))
# Filter for ID column usually used for joins
join_key = [c for c in common_rest_cols if 'id' in c][0]
print(f"7. Join Key (Orders-Restaurants): {join_key}")

# 8. Food type column
# Logic: Look for 'cuisine' in column names
food_col = [c for c in restaurants_df.columns if 'cuisine' in c][0]
print(f"8. Food Type Column: {food_col}")

# 9. User details repetition
# Logic: Check if user counts > 1
is_repeated = merged_df['user_id'].value_counts().max() > 1
print(f"9. User Details Repeated? {'Yes (Multiple times)' if is_repeated else 'No'}")

Data Loaded: Orders (10000, 6)
Data Loaded: Users (3000, 4)
Data Loaded: Restaurants (500, 4)
Success: 'final_food_delivery_dataset.csv' created.

 PART 1: ANALYTICAL SOLUTIONS
1. Highest Gold Revenue City: Chennai
2. Highest AOV Cuisine: Mexican
3. Count of users > 1000 value: 2544 (Corresponds to option '> 2000')
4. Highest Revenue Rating Range: 4.6 – 5.0
5. Highest AOV City (Gold): Chennai
6. Cuisine with lowest distinct restaurants: Chinese
7. Gold Order Percentage: 50%
8. Best Restaurant (<20 orders) from options: Ruchi Foods Chinese
9. Best Revenue Combination: Gold + Italian
10. Highest Revenue Quarter: Q3

 PART 2: NUMERICAL CALCULATIONS
1. Total Gold Orders: 4987
2. Hyderabad Revenue: 1889367
3. Distinct Users: 2883
4. Gold AOV: 797.15
5. Orders (Rating >= 4.5): 3374
6. Orders in Top Gold City (Chennai): 1337

 PART 3: LOGICAL DERIVATION (FILL IN BLANKS)
1. Join Column (Orders-Users): user_id
2. Restaurant Data Format: SQL (Deduced from file content parsing)
3. Total Rows in M